# Repressilator
Example to test handling rational functions (such as hill functions) in Oscar.

Reference: [repressilator.html](https://biocircuits.github.io/chapters/09_repressilator.html)

In [461]:
using CRNAnalysis
using Catalyst
using Oscar
import Symbolics

In [224]:
function repressilator(; n::Int, name=:Repressilator)
    t = default_t()
    @reaction_network $name begin
        @species X(t) Y(t) Z(t)
        hillr(Z, β, 1, $n), 0 --> X
        hillr(X, β, 1, $n), 0 --> Y
        hillr(Y, β, 1, $n), 0 --> Z
        1, X --> 0
        1, Y --> 0
        1, Z --> 0
    end
end
rn = repressilator(n=2)
nlsys = convert(NonlinearSystem, rn)

Model Repressilator:
Equations (3):
  3 standard: see equations(Repressilator)
Unknowns (3): see unknowns(Repressilator)
  X(t)
  Y(t)
  Z(t)
Parameters (1): see parameters(Repressilator)
  β

Steps:
- check if mass-action/polynomial
  - if all reactions are, continue as normal
- if not, check for rational functions
  - convert to polynomials
    - factor out denominator of rational functions (LHS = 0)
- otherwise, error

In [243]:
vars = Catalyst.unknowns(nlsys)
params = parameters(nlsys)
eqns = equations(nlsys)
polyn_rhs = expand_rational_equation.(eqns)

3-element Vector{Num}:
 -X(t) + β - X(t)*(Z(t)^2)
 -Y(t) + β - Y(t)*(X(t)^2)
 -Z(t) + β - (Y(t)^2)*Z(t)

In [249]:
beta = 10
polyn_rhs = Symbolics.substitute(polyn_rhs, Dict(params[1] => beta))

3-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 10 - X(t) - X(t)*(Z(t)^2)
 10 - Y(t) - Y(t)*(X(t)^2)
 10 - Z(t) - (Y(t)^2)*Z(t)

In [250]:
# Oscar Ideal
I = polynomial_ideal(polyn_rhs, vars, params)

Ideal generated by
  -X*Z^2 - X + 10
  -X^2*Y - Y + 10
  -Y^2*Z - Z + 10

In [251]:
G = Oscar.groebner_basis(I, complete_reduction=true)

Gröbner basis with elements
  1: 100*X*Y + 101*Y^2 - 100*X*Z - 101*Z^2 - 10*X + 10*Z
  2: 101*X^2 - 101*Y^2 + 100*X*Z - 100*Y*Z + 10*Y - 10*Z
  3: 10*Y*Z^2 - 101*Y^2 + 100*X*Z + Z^2 + 10*X + 10*Y - 10*Z - 100
  4: X*Z^2 + X - 10
  5: Y^2*Z + Z - 10
  6: 101*Y^3 - 101*Z^3 - 10*Y^2 - 10*X*Z + 10*Y*Z + 10*Z^2 + 100*X + 101*Y - 201*Z
  7: 101*Z^4 - 10*Z^3 - 101*Y^2 + 202*Z^2 - 1000*Y - 20*Z + 100
with respect to the ordering
  degrevlex([X, Y, Z])

In [252]:
# eliminate Y and Z to get one equation
# since the equations are symmetric this describes the full system
x = gens(base_ring(I))
EI = eliminate(I, x[2:3])
p = to_univariate(EI[1])

101*X^9 - 10*X^8 + 404*X^7 - 40*X^6 + 806*X^5 - 2060*X^4 + 804*X^3 - 4040*X^2 + 10301*X - 102010

In [275]:
@show roots(p)
@show p(2)

# https://biocircuits.github.io/chapters/09_repressilator.html
fixed_point(x0, β, n) = x0*(1 + x0^n) - β
fixed_point(2, 10, 2)

roots(p) = AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}[2]
p(2) = 0


0

The root found for polynomial $p$ matches the formula given analytically. So we know that the Groebner basis method works.

In [292]:
N = 5
beta = 2
rn = repressilator(n=N)
I = let 
    nlsys = convert(NonlinearSystem, rn)
    vars = Catalyst.unknowns(nlsys)
    params = parameters(nlsys)
    eqns = equations(nlsys)
    polyn_rhs = expand_rational_equation.(eqns)
    polyn_rhs = Symbolics.substitute(polyn_rhs, Dict(params[1] => beta))
    polynomial_ideal(polyn_rhs, vars, params)
end

Ideal generated by
  -X*Z^5 - X + 2
  -X^5*Y - Y + 2
  -Y^5*Z - Z + 2

In [285]:
x = gens(base_ring(I))
EI = eliminate(I, x[2:3])
q = to_univariate(EI[1])

33*X^126 - 2*X^125 + 825*X^121 - 50*X^120 + 9900*X^116 - 600*X^115 + 75900*X^111 - 4600*X^110 + 417450*X^106 - 25300*X^105 + 1753450*X^101 - 106580*X^100 + 5847500*X^96 - 360600*X^95 + 15893500*X^91 - 1022200*X^90 + 35874375*X^86 - 2527950*X^85 + 68193375*X^81 - 5636350*X^80 + 110359960*X^76 - 11519280*X^75 + 153449400*X^71 - 21625200*X^70 + 185088300*X^66 - 37357400*X^65 + 196424300*X^61 - 60029400*X^60 + 187945400*X^56 - 90617200*X^55 + 168508440*X^51 - 127816240*X^50 + 148819775*X^46 - 166889150*X^45 + 136487175*X^41 - 203753550*X^40 + 133482300*X^36 - 236199800*X^35 + 132109900*X^31 - 252885400*X^30 + 122802890*X^26 - 242205460*X^25 + 110197450*X^21 - 219585300*X^20 + 96667900*X^16 - 193188600*X^15 + 68289900*X^11 - 136560600*X^10 + 29648825*X^6 - 59296050*X^5 + 39135425*X - 78270786

In [299]:
@show roots(q)[]
@show fixed_point(1, 2, N)

(roots(q))[] = 1
fixed_point(1, 2, N) = 0


0

In [309]:
f = p / 101

X^9 - 10//101*X^8 + 4*X^7 - 40//101*X^6 + 806//101*X^5 - 2060//101*X^4 + 804//101*X^3 - 40*X^2 + 10301//101*X - 1010

In [307]:
monomial_to_newton!(f.coeffs, roots(f))

In [328]:
x = gen(parent(f))
g = f / (x - 2)

X^8 + 192//101*X^7 + 788//101*X^6 + 1536//101*X^5 + 3878//101*X^4 + 5696//101*X^3 + 12196//101*X^2 + 20352//101*X + 505

In [419]:
function get_complex_roots(f)
    # need to map coefficients to sensible number types first
    ff = map_coefficients(c -> Rational(c.num.content_num, c.den.content_num), f)
    rs = roots(complex_field(), g)
    ComplexF64.(rs)
end

rs = get_complex_roots(f)

9-element Vector{ComplexF64}:
                 2.0 + 0.0im
 -1.8345521418287438 - 0.9703309566605561im
 -1.8345521418287438 + 0.9703309566605561im
  1.4033297112175989 - 1.4591532600809507im
  1.4033297112175989 + 1.4591532600809507im
                -1.0 + 2.0im
                -1.0 - 2.0im
  0.4807273811061943 + 2.343169386176345im
  0.4807273811061943 - 2.343169386176345im

In [422]:
@show fixed_point(-1 + 2im, 10, 2)
@show fixed_point(-1 - 2im, 10, 2)

fixed_point(-1 + 2im, 10, 2) = 0 + 0im
fixed_point(-1 - 2im, 10, 2) = 0 + 0im


0 + 0im

# Goodwin model

In [477]:
function goodwin_model(; n::Int, name=:GoodwinModel)
    t = default_t()
    goodwin = @reaction_network $name begin
        @species R(t) P(t) X(t)
        @parameters k1 k2 k3 k4 k5 k6 K
        hillr(X, k1, K, $n), 0 => R
        k2, R --> 0
        k3*R, 0 --> P
        k4, P --> 0
        k5*P, 0 --> X
        k6, X --> 0
    end
end
goodwin = goodwin_model(n=5)

Model GoodwinModel:
Unknowns (3): see unknowns(GoodwinModel)
  R(t)
  P(t)
  X(t)
Parameters (7): see parameters(GoodwinModel)
  k1
  k2
  k3
  k4
  ⋮

In [478]:
pmap = [rn.k1 => 10, rn.k2 => 1, rn.k3 => 10, rn.k4 => 1, rn.k5 => 10, rn.k6 => 1, rn.K => 10]
pmap = [k => v//10 for (k, v) in pmap]

I = let 
    nlsys = convert(NonlinearSystem, goodwin)
    vars = Catalyst.unknowns(nlsys)
    params = parameters(nlsys)
    eqns = equations(nlsys)
    eqns = Symbolics.substitute(eqns, Dict(pmap))
    # polyn_rhs = expand_rational_equation.(eqns) #FIXME
    polyn_rhs = map(eqns) do eq
        ff=implicit_form(eq)
        denoms = Num[Symbolics.denominator(term)
            for term in Symbolics.terms(ff)
            if is_rational_function(term)
        ]
        new_f = ff * prod(denoms)
        Symbolics.simplify(new_f, expand=true)
    end
    polynomial_ideal(polyn_rhs, vars, params)
end

Ideal generated by
  -1//10*R*X^5 - 1//10*R + 1
  R - 1//10*P
  P - 1//10*X

In [479]:
x = gens(base_ring(I))
EI = eliminate(I, x[2:3])
q = to_univariate(EI[1])

10000000000*R^6 + R - 10